## Extracting a subset of information from the Twitter JSON Data

In this example, we read a collection of timestamped Twitter data in JSON format collected using the Twitter Streaming API to collect tweets during the period of November 20 to December 5, 2018, searching on the hashtag 'climatechange'. The data in JSON format comprises ~3.13 Gb of Twitter data in uncompressed text format. Below we list some basic statistics of the #climatechange dataset.

* Start: Tue Nov 20 00:07:52 +0000 2018
* End: Wed Dec 05 18:38:47 +0000 2018
* 457294 Total Tweets
* 138191 Original Tweets
* 319103 Retweets
* 22580 Unique User Screen Names
* 44127 Unique Retweets

<b>NOTE</b>: Since Twitter imposes a limit of 50,000 full tweets for the number of tweets we are allowed to share in a single day, we do not actually provide the original JSON data for this example. However, the code below can be generalized to any Twitter data collected with the Streaming API and saved as JSON. We provide a subset of the JSON data (the two .json files provided in the data/twitter/ subdirectory containing 44,910 tweets) for you to work with in this lesson. Since we also provide the full dataset of 457,294 tweets in reduced format (i.e. the file in the data subdirectory called 'climatechange_tweets_all.csv', see below), we want to be sure when doing this lesson that you don't overwrite that file, so for this lesson we will name our output file 'climatechange_tweets_sample.csv'. 

As you have already seen, a tweet may consist of more than 40 fields. For our analysis we are only interested in a subset of fields (15). This should greatly reduce the resulting file size. You are of course encouraged to adapt this code to suit your specific needs if you want more (or less) than these 15 fields when analyzing your own data.

The 15 fields we will be extracting are:

<ul>
    <li>id</li>
    <li>created_at</li>
    <li>lang</li>
    <li>screen_name (user)</li>
    <li>created_at (user)</li>
    <li>id (user)</li>
    <li>followers_count (user)</li>
    <li>friends_count (user)</li>
    <li>utc_offset (user)</li>
    <li>time_zone (user)</li>
    <li>text</li>
</ul>

and if the tweet is a retweet...
<ul>
    <li>retweeted_status</li>
    <li>id</li>
    <li>screen_name (user)</li>
    <li>created_at (user)</li>
</ul>




## Import Packages

In [1]:
# Load packages
import glob, os
import csv
import sys
import json
import datetime
import pandas as pd
from pprint import pprint

## Open JSON, Parse Data, Save as CSV

In the sample JSON files provided,  we read the JSON data line-by-line using the <code>json.loads()</code> command. This is because each tweet is an individual line in the JSON text file. The comments in the code below describe the purpose of each section of code. Most but not all tweets are retweets (319,103). Some tweets are original tweets (138,191). Original tweets do not contain the same data as retweets. In particular, original tweets do not include the 'retweeted_status' object. As a result, if we want to include both original tweets and retweets in our dataframe, we need a way to handle the extra data found only in retweets. We do this with a check on the 'retweeted_status' key in the tweet data and deal with the four parameters (retweeted_status, retweet_id, retweet_user_screen_name, retweet_user_id) accordingly.

We also include an optional counter which prints a statement as each 10,000th tweet is read, making it easy to track progress with large files without doing too much printing.

The total time for this process was roughly 90 seconds. Saving this reduced data set in a CSV file will enable us to quickly load the data in a Pandas dataframe, which is useful for further analysis.

In [2]:
# Print start time at start and end time at end
print("Start:" + str(datetime.datetime.now()))

# Open CSV output files for reading and writing
input_dir = "../data/twitter/"
output_dir = "../data/twitter/"
hashtag = "climatechange"

# Open main twitter data CSV file and write header row
output_file = output_dir + hashtag + "_tweets_sample.csv"
f_out = open(output_file, 'w', encoding='utf-8')
rowwriter = csv.writer(f_out, delimiter=',', lineterminator='\n')
outputrow = ['tweet_id','tweet_created_at','language','user_screen_name','user_created_at','user_id','followers_count','friends_count','time_zone','utc_offset','retweeted_status','retweet_id','retweet_user_screen_name','retweet_user_id','text']
rowwriter.writerow(outputrow)

# Define variables
inc = 0

files = glob.glob(input_dir + '*.json')
for file in files:
    with open(file, 'r', encoding='utf-8') as f:
        print("Working on file:" + file)
        for line in f:
            tweet = json.loads(line)
            if 'user' in tweet:
                
                # Set standard variables equal to tweet data
                tweet_id = tweet['id']
                tweet_created_at = tweet['created_at']
                text = tweet['text'].replace('\r', ' ').replace('\n', ' ')
                language = tweet['lang']
                user_screen_name = tweet['user']['screen_name']
                user_created_at = tweet['user']['created_at']
                user_id = tweet['user']['id']
                followers_count = tweet['user']['followers_count']
                friends_count = tweet['user']['friends_count']
                utc_offset = tweet['user']['utc_offset']
                time_zone = tweet['user']['time_zone']

                # Check if a retweet else original tweet
                if 'retweeted_status' in tweet:
                    retweeted_status = 1
                    retweet_id = tweet['retweeted_status']['id']
                    retweet_user_screen_name = tweet['retweeted_status']['user']['screen_name']
                    retweet_user_id = tweet['retweeted_status']['user']['id']
                else:
                    retweeted_status = 0
                    retweet_id = "None"
                    retweet_user_screen_name = "None"
                    retweet_user_id = "None"

                # Write to main output file
                outputrow = [str(tweet_id), tweet_created_at, language, user_screen_name, user_created_at, str(user_id), str(followers_count), str(friends_count), time_zone, utc_offset, str(retweeted_status), str(retweet_id), retweet_user_screen_name, str(retweet_user_id), text] 
                rowwriter.writerow(outputrow)

                inc += 1
                # Optional counter increments variables to track progress, useful for very large files.
                if inc%10000 == 0:
                    print(inc)

# Close the output file
f_out.close()
print("End:" + str(datetime.datetime.now()))
print("Total number of tweets: {}".format(inc))

Start:2019-12-09 16:15:59.041345
Working on file:../data/twitter/climatechange_2018_11_26_17_19_15_679824.json
10000
20000
Working on file:../data/twitter/climatechange_2018_11_27_11_30_06_972631.json
30000
40000
End:2019-12-09 16:16:03.849446
Total number of tweets: 44910


## Tweak the CSV file to accomodate timestamps

The CSV file we just wrote is a much smaller distillation of the data of interest to us, but we'll want to tweak it a little bit so that we can work with it in pandas more easily.  

Two of the fields in the CSV file represent timestamps: <code>tweet_created_at</code> and <code>user_created_at</code>.  The <code>pd.read_csv</code> function can be passed an additional argument indicating that particular columns should be parsed as dates (datetimes), instead of plain strings.  Unfortunately, the Twitter data that we extracted from the JSON file is in a format that is not the standard datetime format, e.g., <code>Thu Nov 29 19:22:55 +0000 2018</code>.  Pandas has a function named <code>to_datetime</code> that can not only convert strings to datetime objects, but can infer datetimes from a number of different formats.  That inference can be rather slow for a large file, however.

Fortunately, the <code>pd.to_datetime</code> function can be provided with a specific format string, so that it does not need to infer a format.  By providing an explicit format string to the conversion function for our tweet timestamps, the conversion is quite fast (a few seconds). Format strings are based upon the Python strftime directives (link). If we do not provide a format hint of this sort, the conversion takes more than a minute for each function call.  In that code that follows, we:

* read the CSV file into pandas
* convert two of the fields that are timestamps to datetime objects with a specific format string
* write out a new CSV file with the reformatted data


In [3]:
output_dir = "../data/twitter/"
tweet_df = pd.read_csv(output_dir + 'climatechange_tweets_sample.csv')
tweet_df.tweet_created_at = pd.to_datetime(tweet_df.tweet_created_at, format='%a %b %d %H:%M:%S +0000 %Y')
tweet_df.user_created_at = pd.to_datetime(tweet_df.user_created_at, format='%a %b %d %H:%M:%S +0000 %Y')
tweet_df.to_csv(output_dir + 'climatechange_tweets_fixed.csv', index=False)

## Working further with the reformatted CSV file

Having tweaked the datetime formats, if we want to read our new CSV file into a pandas dataframe, we can augment the <code>pd.read_csv</code> call to specify which columns to parse as dates, using the <code>parse_dates</code> option.  If we had not previously altered the datetime formats, this call to <code>pd.read_csv</code> would be very slow due to the new to do datetime format inference.  Now that we have fixed the formats, the date parsing proceeds quickly.  The <code>pd.read_csv</code> function, however, does not allow one to specify a format string, which is why we needed to do the conversion with the <code>pd.to_datetime</code> function as above.  Here's what the new read_csv function call looks like:

In [4]:
tweet_df = pd.read_csv(output_dir + 'climatechange_tweets_fixed.csv', \
                       parse_dates=['tweet_created_at', 'user_created_at'])

And finally, we can view our resulting dataframe. 

In [5]:
tweet_df

,tweet_id,tweet_created_at,language,user_screen_name,user_created_at,user_id,followers_count,friends_count,time_zone,utc_offset,retweeted_status,retweet_id,retweet_user_screen_name,retweet_user_id,text
0,1067211197412388864,2018-11-27 00:19:11,en,MarieCo92176893,2014-05-06 01:55:58,2479088160,1874,260,NaN,NaN,1,1067192127124164612,DocsEnvAus,1542665564,RT @DocsEnvAus: @susanprescott88 paediatrician...
1,1067211198842716162,2018-11-27 00:19:12,en,FocusOnQuality,2010-09-14 22:06:33,190809029,1745,2569,NaN,NaN,1,1067163637847085058,RepLoisFrankel,1077121945,"RT @RepLoisFrankel: Here’s the thing, @realDon..."
2,1067211205083709440,2018-11-27 00:19:13,en,JudyOsburn8,2018-07-21 03:28:19,1020510759384563712,158,198,NaN,NaN,1,1067149009867878400,jessphoenix2018,842550390818201600,RT @jessphoenix2018: If you're not willing to ...
3,1067211213409566722,2018-11-27 00:19:15,en,Rajivsheoran5,2018-11-23 13:57:04,1065967469607747587,1,16,NaN,NaN,1,1067051467083669504,UNinIndia,1856588299,RT @UNinIndia: Climate change affects us all! ...
4,1067211218258018304,2018-11-27 00:19:16,en,PaDenys,2015-02-08 20:57:34,3013785847,686,764,NaN,NaN,0,None,None,None,@WasilewskiTomek #ClimateChange in action. 4 c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44905,1067782537362227200,2018-11-28 14:09:30,en,NeverForget56,2015-12-15 22:16:53,4496345055,2685,3831,NaN,NaN,0,None,None,None,#DemocratsAreDangerous #ClimateChange is a hoax.
44906,1067782538742124546,2018-11-28 14:09:30,en,jbriiicpa,2009-05-17 19:11:06,40716404,309,559,NaN,NaN,1,1067763679486197761,Janis_Kelly,18100204,RT @Janis_Kelly: The #USGCRP warns that #Clima...
44907,1067782545465581569,2018-11-28 14:09:31,en,RedPillDetox,2017-12-10 19:08:41,939934915629670411,4454,4484,NaN,NaN,1,1067781725206515717,ArtMusicLife,30296735,RT @ArtMusicLife: I. Just. Can’t. #Resist #Cli...
44908,1067782548443602945,2018-11-28 14:09:32,en,2ysur2ysub,2009-04-15 05:01:59,31345344,12255,11341,NaN,NaN,1,921762776133046273,RichardAngwin,459269265,RT @RichardAngwin: Trump's energy policy #Sat...


In the subsequent material (found in the notebook 'climatechange_tweet_timelines.ipynb') will use the CSV file containing the entire dataset of 457,294 tweets for further analysis and visualization, which are stored in a file named 'climatechange_tweets_all.csv'.  The date reformatting described above has already been performed for that data file.